In [1]:
from datasets import load_dataset
from transformer_lens import HookedTransformer
from sae_lens import SAE
import torch

from transformer_lens.utils import tokenize_and_concatenate
from huggingface_hub import login

/n/home04/cfang/.conda/envs/sae/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from configs import huggingface_token

In [3]:
login(token=huggingface_token)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
release = "gemma-scope-2b-pt-res-canonical"
sae_id = "layer_20/width_16k/canonical"
sae = SAE.from_pretrained(release, sae_id)[0]

In [6]:
sae.cfg.hook_name

'blocks.20.hook_resid_post'

In [7]:
model = HookedTransformer.from_pretrained("gemma-2-2b", device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  6.68it/s]


Loaded pretrained model gemma-2-2b into HookedTransformer


# Loading and running on Pile

In [8]:
dataset = load_dataset(
    path="NeelNanda/pile-10k",
    split="train",
    streaming=False,
)

In [9]:
token_dataset = tokenize_and_concatenate(
    dataset=dataset,  # type: ignore
    tokenizer=model.tokenizer,  # type: ignore
    streaming=True,
    max_length=sae.cfg.context_size,
    add_bos_token=sae.cfg.prepend_bos,
)

In [11]:
batch_tokens = token_dataset[:32]["tokens"]
with torch.no_grad():
    _, cache = model.run_with_cache(
        batch_tokens,
        prepend_bos=True,
        names_filter=lambda name: name == "blocks.20.hook_resid_post"
    )

OutOfMemoryError: CUDA out of memory. Tried to allocate 31.25 GiB. GPU 0 has a total capacity of 79.22 GiB of which 1.25 GiB is free. Including non-PyTorch memory, this process has 77.95 GiB memory in use. Of the allocated memory 77.14 GiB is allocated by PyTorch, and 156.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model_activations = cache[sae.cfg.hook_name]
feature_acts = sae.encode(model_activations)
sae_out = sae.decode(feature_acts)